In [1]:
# Install the required version of transformers
!pip install -U transformers==4.40.2

# Import necessary libraries
import torch
from transformers import MegaConfig, MegaForSequenceClassification, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import os


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 63.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.3
    Uninstalling transformers-4.46.3:
      Successfully uninstalled transformers-4.46.3


In [2]:
# Define the file paths
train_file = '/kaggle/input/lra-listops-reduced/train_d20s.tsv'
test_file = '/kaggle/input/lra-listops-reduced/test_d20s.tsv'

# Load the data
train_df = pd.read_csv(train_file, sep='\t', header=0)
test_df = pd.read_csv(test_file, sep='\t', header=0)
# Preview the data
print(train_df.head())

# Load the data
train_df = pd.read_csv(train_file, sep='\t', header=0)
test_df = pd.read_csv(test_file, sep='\t', header=0)
# Preview the data
print(train_df.head())
print(train_df.columns)


   Target                                             Source
0       6  ( ( ( ( ( [MAX ( ( ( ( ( ( [MED 4 ) 6 ) 6 ) 0 ...
1       7   ( ( ( ( [SM ( ( ( [MED 6 ) 5 ) ] ) ) 1 ) 1 ) ] )
2       4                 ( ( ( ( ( [MAX 3 ) 4 ) 3 ) 3 ) ] )
3       0  ( ( ( ( ( [MIN 0 ) 0 ) ( ( ( [MAX 4 ) ( ( ( ( ...
4       9  ( ( ( ( ( ( [SM ( ( ( ( [MIN 5 ) ( ( ( [MAX ( ...
   Target                                             Source
0       6  ( ( ( ( ( [MAX ( ( ( ( ( ( [MED 4 ) 6 ) 6 ) 0 ...
1       7   ( ( ( ( [SM ( ( ( [MED 6 ) 5 ) ] ) ) 1 ) 1 ) ] )
2       4                 ( ( ( ( ( [MAX 3 ) 4 ) 3 ) 3 ) ] )
3       0  ( ( ( ( ( [MIN 0 ) 0 ) ( ( ( [MAX 4 ) ( ( ( ( ...
4       9  ( ( ( ( ( ( [SM ( ( ( ( [MIN 5 ) ( ( ( [MAX ( ...
Index(['Target', 'Source'], dtype='object')


In [3]:
def clean_text(text):
    return ''.join(str(text).split())

# Apply the cleaning function to the 'Source' column
#train_df['Source'] = train_df['Source'].apply(clean_text)
#test_df['Source'] = test_df['Source'].apply(clean_text)


In [4]:

# Remove any possible header rows included as data
train_df = train_df[train_df['Target'] != 'Target']
test_df = test_df[test_df['Target'] != 'Target']

# Convert labels to integers
train_df['Target'] = train_df['Target'].astype(int)
test_df['Target'] = test_df['Target'].astype(int)
print(train_df.head())

print(len(train_df['Source'][0]))
print(len(train_df['Source'][1]))
print(len(train_df['Source'][2]))
print(len(train_df['Source'][3]))
print(len(train_df['Source'][4]))
print(len(train_df['Source'][5]))
print(len(train_df['Source'][8]))

   Target                                             Source
0       6  ( ( ( ( ( [MAX ( ( ( ( ( ( [MED 4 ) 6 ) 6 ) 0 ...
1       7   ( ( ( ( [SM ( ( ( [MED 6 ) 5 ) ] ) ) 1 ) 1 ) ] )
2       4                 ( ( ( ( ( [MAX 3 ) 4 ) 3 ) 3 ) ] )
3       0  ( ( ( ( ( [MIN 0 ) 0 ) ( ( ( [MAX 4 ) ( ( ( ( ...
4       9  ( ( ( ( ( ( [SM ( ( ( ( [MIN 5 ) ( ( ( [MAX ( ...
73
48
34
271
501
577
2542


In [5]:
# Compute the lengths of the original sequences
seqLengths= train_df['Source'].apply(lambda x: len(x))

# Describe the sequence lengths
print("Training data sequence lengths:")
print(seqLengths.describe())

print(seqLengths.head())





Training data sequence lengths:
count    90000.000000
mean       277.778200
std        491.629979
min          1.000000
25%         49.000000
50%         99.000000
75%        259.000000
max       7593.000000
Name: Source, dtype: float64
0     73
1     48
2     34
3    271
4    501
Name: Source, dtype: int64


In [6]:
import pandas as pd

# Filter the training and test datasets based on the condition
train_df = train_df[train_df['Source'].apply(lambda x: len(x)) < 1024]
test_df = test_df[test_df['Source'].apply(lambda x: len(x)) < 1024]

# Randomly sample 10% of the filtered train data
train_df = train_df.sample(frac=1, random_state=42)  # Set random_state for reproducibility

# Check the filtered dataframe
seqLengths = train_df['Source'].apply(lambda x: len(x))

# Describe the sequence lengths
print("Training data sequence lengths:")
print(seqLengths.describe())

print(seqLengths.head())


Training data sequence lengths:
count    84162.000000
mean       170.649367
std        194.364638
min          1.000000
25%         48.000000
50%         88.000000
75%        207.000000
max       1023.000000
Name: Source, dtype: float64
33140     34
16844    483
50823     55
73017    292
5116     121
Name: Source, dtype: int64


In [7]:
# Initialize the tokenizer
#tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')


In [8]:
# Save the sequences to a text file for tokenizer training
with open("listops_sequences.txt", "w") as f:
    for sequence in train_df["Source"]:
        f.write(sequence + "\n")


In [9]:
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace

# Initialize a WordLevel tokenizer
tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))

# Set the pre-tokenization strategy
tokenizer.pre_tokenizer = Whitespace()

# Prepare a trainer with special tokens
trainer = WordLevelTrainer(special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"])

# Train the tokenizer on your text file
tokenizer.train(["listops_sequences.txt"], trainer)

# Save the tokenizer
tokenizer.save("custom_tokenizer.json")


In [10]:
from transformers import PreTrainedTokenizerFast

# Load the custom tokenizer
tokenizer = PreTrainedTokenizerFast(tokenizer_file="custom_tokenizer.json")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
print(tokenizer)

PreTrainedTokenizerFast(name_or_path='', vocab_size=23, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'pad_token': '[PAD]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}


In [11]:
class LRADataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels.astype(int)  # Ensure labels are integers
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item]

        # Tokenize and encode the text
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


In [12]:
def create_data_loader(df, tokenizer, max_len, batch_size):
    dataset = LRADataset(
        texts=df['Source'].to_numpy(),
        labels=df['Target'].to_numpy(),
        tokenizer=tokenizer,
        max_len=max_len
    )

    return DataLoader(
        dataset,
        batch_size=batch_size,
        num_workers=10
    )

# Parameters
MAX_LEN = 1024
BATCH_SIZE = 128

# Create data loaders
train_data_loader = create_data_loader(train_df, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(test_df, tokenizer, MAX_LEN, BATCH_SIZE)


/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [13]:
num_labels = train_df["Source"].nunique()
print(num_labels)
print(tokenizer.vocab_size)


84162
23


In [14]:
# Number of unique labels


# Configure the MEGA model
config = MegaConfig(
    vocab_size=tokenizer.vocab_size,
    hidden_size=16,
    num_hidden_layers=4,
    num_labels=10,
    max_positions=MAX_LEN,
    bidirectional=True,
    is_decoder=False,
    use_cache=False,
    activation_function='silu',  # Match activation-fn parameter
    attention_activation_function='softmax',  # Match attention-activation-fn parameter
    norm_type='layernorm',  # Match norm-type parameter
    dropout=0.1,  # Match dropout parameter
    attention_dropout=0.0,  # Match attention-dropout parameter
    weight_decay=0.01,

)


model = MegaForSequenceClassification(config)


In [15]:
from transformers import AdamW, get_linear_schedule_with_warmup
import torch.nn as nn


# Move the model to GPU(s)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs")
    model = nn.DataParallel(model)  #
model = model.to(device)


# Optimizer and scheduler
EPOCHS = 5
optimizer = AdamW(model.parameters(), lr=2e-4)
total_steps = len(train_data_loader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=total_steps // 10,
    num_training_steps=total_steps
)

# Loss function
loss_fn = torch.nn.CrossEntropyLoss().to(device)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [16]:
def train_epoch(
    model,
    data_loader,
    loss_fn,
    optimizer,
    device,
    scheduler
):
    model = model.train()
    losses = []
    correct_predictions = 0

    for batch in data_loader:
        input_ids = batch["input_ids"].to(device, non_blocking=True)
        attention_mask = batch["attention_mask"].to(device, non_blocking=True)
        labels = batch["labels"].to(device, non_blocking=True)

        outputs = model(
            input_ids=input_ids.cuda(),
            attention_mask=attention_mask.cuda(),
            labels=labels.cuda()
        )
        loss = outputs.loss
        logits = outputs.logits
        _, preds = torch.max(logits, dim=1)

        correct_predictions += torch.sum(preds == labels)
        losses.append(loss.item())

        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    return correct_predictions.double() / len(data_loader.dataset), np.mean(losses)

def eval_model(
    model,
    data_loader,
    loss_fn,
    device
):
    model = model.eval()
    losses = []
    correct_predictions = 0

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch["input_ids"].to(device, non_blocking=True)
            attention_mask = batch["attention_mask"].to(device, non_blocking=True)
            labels = batch["labels"].to(device, non_blocking=True)

            outputs = model(
                input_ids=input_ids.cuda(),
                attention_mask=attention_mask.cuda(),
                labels=labels.cuda()
            )
            loss = outputs.loss
            logits = outputs.logits
            _, preds = torch.max(logits, dim=1)

            correct_predictions += torch.sum(preds == labels)
            losses.append(loss.item())

    return correct_predictions.double() / len(data_loader.dataset), np.mean(losses)


In [17]:
history = {
    'train_acc': [],
    'train_loss': [],
    'val_acc': [],
    'val_loss': []
}

for epoch in range(EPOCHS):
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    print('-' * 10)

    train_acc, train_loss = train_epoch(
        model,
        train_data_loader,
        loss_fn,
        optimizer,
        device,
        scheduler
    )

    print(f'Train loss {train_loss} accuracy {train_acc}')

    val_acc, val_loss = eval_model(
        model,
        test_data_loader,
        loss_fn,
        device
    )

    print(f'Val   loss {val_loss} accuracy {val_acc}')

    history['train_acc'].append(train_acc.cpu().numpy())
    history['train_loss'].append(train_loss)
    history['val_acc'].append(val_acc.cpu().numpy())
    history['val_loss'].append(val_loss)


Epoch 1/5
----------


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling pa

Train loss 2.168084220502152 accuracy 0.17078966754592334


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Val   loss 2.0057558803944975 accuracy 0.2056124626547162
Epoch 2/5
----------


...

Val   loss 1.8459887134062278 accuracy 0.2759282970550576
Epoch 5/5
----------


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Train loss 1.8552908792321805 accuracy 0.2743399634039115


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Val   loss 1.835294554362426 accuracy 0.279449423815621


In [18]:
# Specify the directory to save the model
output_dir = './mega_model_listops_reduced_best_scheduled/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Save the trained model and tokenizer
model_to_save = model.module if hasattr(model, 'module') else model
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"Model saved to {output_dir}")


Model saved to ./mega_model_listops_reduced_best_scheduled/


In [19]:
import torch
from transformers import MegaForSequenceClassification, PreTrainedTokenizerFast

# Specify the directory where the model and tokenizer are saved
output_dir = './mega_model_listops_best/'

# Load the tokenizer
tokenizer = PreTrainedTokenizerFast.from_pretrained(output_dir)

# Load the model
model = MegaForSequenceClassification.from_pretrained(output_dir)

# Move the model to the appropriate device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


OSError: Incorrect path_or_model_id: './mega_model_listops_best/'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [ ]:
# Iterate over sequences to tokenize them individually
for sequence in sequences:
    tokens = tokenizer.tokenize(sequence)
    print("Tokens for sequence:", sequence)
    print("Tokens:", tokens)





# Correct way to batch-encode sequences
encoding = tokenizer(
    sequences,
    add_special_tokens=True,
    max_length=model.config.max_length,  # Correct field for model's max length
    padding='max_length',
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt'
)

# Move tensors to device
input_ids = encoding['input_ids'].to(device)
attention_mask = encoding['attention_mask'].to(device)

# Print for verification
print("Input IDs:", input_ids)
print("Attention Mask:", attention_mask)


In [ ]:
# Put the model in evaluation mode
model.eval()

# Disable gradient calculations for inference
with torch.no_grad():
    outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
    )

# The outputs contain logits
logits = outputs.logits
print("Input IDs shape:", input_ids.shape)
print("Attention mask shape:", attention_mask.shape)



In [ ]:
# For batched input
predicted_class_idx = torch.argmax(probabilities, dim=-1).cpu()  # Get indices for all examples
print(f"Predicted class indices: {predicted_class_idx.tolist()}")


In [ ]:
from transformers import AutoTokenizer
from torch.utils.data import DataLoader


sample_texts = [
    "( MIN 3 5 6 9 7 3 )",
    "( MAX 2 4 6 )",
    "( SUM 1 2 3 4 5 )"
]
sample_labels = [0, 1, 2]  # Sample class labels

# Dataset parameters
max_len = 16

# Create the dataset
dataset = LRADataset(texts=sample_texts, labels=np.array(sample_labels), tokenizer=tokenizer, max_len=max_len)

# Inspect the first item in the dataset
sample_item = dataset[0]

print("Input IDs:", sample_item['input_ids'])
print("Attention Mask:", sample_item['attention_mask'])
print("Labels:", sample_item['labels'])

# Use a DataLoader to batch and inspect multiple items
dataloader = DataLoader(dataset, batch_size=2)

# Get a batch
batch = next(iter(dataloader))

print("\nBatch Input IDs:\n", batch['input_ids'])
print("Batch Attention Mask:\n", batch['attention_mask'])
print("Batch Labels:\n", batch['labels'])
